In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/NIDHI/

/content/drive/MyDrive/NIDHI


In [3]:
!nvidia-smi

Tue Jul 11 05:08:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/NIDHI


## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [5]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [6]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="TZtMC8N2bbQzE5PlVKlg")
# project = rf.workspace("tihomv-xku0k").project("skindetect_")
# dataset = project.version(1).download("yolov8")

In [ ]:
# !yolo train data=/content/drive/MyDrive/NIDHI/datasets/skindetect_seg/data.yaml model=yolov8s-seg.pt epochs=10 lr0=0.01

In [28]:
from ultralytics import YOLO
import cv2
from random import randint

# Load a model
model = YOLO('/content/drive/MyDrive/NIDHI/runs/segment/train13/weights/best.pt')  # pretrained YOLOv8n model
def seg(sour):
  try:
    # print("sour")
    results = model(sour,retina_masks=True,boxes=True,save=False)  # return a list of Results objects
    result = results[0]
    seg = result.masks.segments[0]

    # img = cv2.imread(image_path)
    img = sour
    h,w = img.shape[:2]
    seg *= [w,h] # Unscale
    cv2.polylines(img, [seg.astype('int')], True, (randint(0,255),randint(0,255),randint(0,255)), 2) # Draw Poly Lines
    overlay = img.copy()
    cv2.fillPoly(overlay, [seg.astype('int')], (randint(0,255),randint(0,255),randint(0,255)), cv2.LINE_AA) # Draw area
    alpha = 0.4
    image_new = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)
    return img
    # return image_new

    # cv2.imwrite("/content/drive/MyDrive/NIDHI/filename.jpg", image_new)
  except Exception as e:
    print("MOMOM")
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(exc_type, fname, exc_tb.tb_lineno)
    print(e)


In [8]:
!pip install ttach

In [9]:
import time
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image
import numpy as np
import sys

In [35]:
model = YOLO('/content/drive/MyDrive/NIDHI/runs/segment/train13/weights/best.pt')
target_layers = [model.model.model[-2]]
def predict_heat(source):
    try:
        img = source
        rgb_img = source.copy()
        img = np.float32(img) / 255
        cam = EigenCAM(model, target_layers, task='cls')
        grayscale_cam = cam(rgb_img)[0, :, :]
        cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
        alpha = 0.6  # Weight for image1
        beta = 0.4  # Weight for image2
        cam_image1 = seg(source)
        cam_image_f = cv2.addWeighted(cam_image1, alpha, cam_image, beta, 0)
        return cam_image_f
        # return cam_image1
    except Exception as e:
      print("MKFDMKD")
      exc_type, exc_obj, exc_tb = sys.exc_info()
      fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
      print(exc_type, fname, exc_tb.tb_lineno)
      print(e)
      return source


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
image_path = "/content/drive/MyDrive/NIDHI/datasets/skindetect_seg/train/images/"
dir_list = os.listdir("/content/drive/MyDrive/NIDHI/datasets/skindetect_seg/train/images")

for fil in dir_list:
    # print(fil)
    if dir_list.index(fil)%5 ==0:
      # print(dir_list.index(fil),fil)
      image_new = predict_heat(cv2.imread(image_path+fil))
      cv2_imshow(image_new)
      cv2.imwrite("/content/drive/MyDrive/result/segmentation/Without_egin/"+fil, image_new)


In [37]:
from google.colab.patches import cv2_imshow
import cv2
image_path = "/content/drive/MyDrive/NIDHI/datasets/skindetect_seg/train/images/"
dir_list = os.listdir("/content/drive/MyDrive/NIDHI/datasets/skindetect_seg/train/images")

for fil in dir_list:
    # print(fil)
    if dir_list.index(fil)%5 ==0:
      # print(dir_list.index(fil),fil)
      image_new = predict_heat(cv2.imread(image_path+fil))
      cv2_imshow(image_new)
      cv2.imwrite("/content/drive/MyDrive/result/segmentation/With_egin/"+fil, image_new)
      # break


Output hidden; open in https://colab.research.google.com to view.